# Task 2 Notebook: The Bug rebuttals
This is the complete code for our research in task 2 of the Age perception challenge.



# Index
- [1. README](#1)
- [2. Weighted Samples Research](#2)
  - [2.1 Imports](#2-1)
  - [2.2 Data Gathering](#2-2)
  - [2.3 The models](#2-3)
  - [2.4 Training and approach](#2-4)
    - [2.4.1 Phase 1](#2-4-1)
    - [2.4.2 Phase 2](#2-4-2)
    - [2.4.3 Different group weighted sampling](#2-4-3)
      - [2.4.3.1 Original](#2-4-3-1)
      - [2.4.3.2 48 groups](#2-4-3-2)
      - [2.4.3.3 7 Age groups](#2-4-3-3)
- [3. Custom Losses Reasearch](#3)
  - [3.1 Pre-requisites](#3-1)
  - [3.2 Imports](#3-2)
  - [3.3 Data Gathering](#3-3)
  - [3.4 The Task 1 model](#3-4)
  - [3.5 Improving Task 1 results using phase 2 training as a baseline](#3-5)
    - [3.5.1 Re-load data and use sample weights](#3-5-1)
    - [3.5.2 Defining Custom Losses](#3-5-2)
    - [3.5.3 Use sample weights and custom loss to train phase 2](#3-5-3)



<a name='1'></a>
# README
The notebook's purpose is to complement the report for this task. Due to time constraints, we have just organized the notebook so the code is easily found in each section but it is not meant to be executable. This is mostly because we have to come up with different model paths locally in our drives (as the website was down) or for example when we combined our work it was simpler to leave the repeated code there so the context is not lost while reading the report and the code at the same time. 

<a name='2'></a>
# Weighted Samples Research

<a name='2-1'></a>
## Imports

In [ ]:
import h5py
import tensorflow as tf
import random 
import pickle
import cv2
import numpy as np

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

from matplotlib import pyplot as plt

from google.colab import drive

<a name='2-2'></a>
## Data Gathering

### Downloading and decompressing the Appa-Real Age Dataset [(source)](http://chalearnlap.cvc.uab.es/challenge/13/track/13/description/)

- As default, RGB images (cropped faces) are in the range of [0, 255], and labels are in the range of ~0.9 to ~90 (years old).
- The data is divided in train, validation and test set. 
- Matadata is also provided
  - gender: male / female 
  - ethnicity: asian / afroamerican / caucasian
  - facial expression: neutral / slightlyhappy / happy / other


In [ ]:
# downloading the data
#!wget https://data.chalearnlap.cvc.uab.cat/Colab_2021/app_data.zip

with ZipFile('drive/MyDrive/Computer Vision/Task 2/app_data.zip','r') as zip:
   zip.extractall()
   print('Data decompressed successfully')

# removing the .zip file after extraction to clean space
#!rm app_data.zip

Data decompressed successfully


### Loading the train/validation data, and re-scaling the labels to [0..1]
- X_[train,valid,test] = Face images
- Y_[train,valid,test] = Ground truth 
- M_[train,valid,test] = Metadata (gender, ethnicicy, facial expression)

In [ ]:
# loading the train data
X_train = np.load('./data/data_train.npy')
Y_train = np.load('./data/labels_train.npy')
M_train = np.load('./data/meta_data_train.npy')

# loading the validation data
X_valid = np.load('./data/data_valid.npy')
Y_valid = np.load('./data/labels_valid.npy')
M_valid = np.load('./data/meta_data_valid.npy')

# loading the test data
X_test = np.load('./data/data_test.npy')
Y_test = np.load('./data/labels_test.npy')
M_test = np.load('./data/meta_data_test.npy')

# train labels are real numbers, ranging from ~0.9 to ~89 (years old);
# we will re-scale the labels to [0,1] by using a normalization factor of 100,
# assuming there is no sample with age > 100.
Y_train = Y_train/100
Y_valid = Y_valid/100
# Y_test = Y_test/100 # -> we don't normalize the test labels as we will evaluate 
                      # them using the raw data, i.e., the apparent age values

print('Train data size and shape', X_train.shape)
print('Train labels size and shape', Y_train.shape)
print('Train metadata size and shape', M_train.shape)
print('----')
print('Valid data size and shape', X_valid.shape)
print('Valid labels size and shape', Y_valid.shape)
print('Valid metadata size and shape', M_valid.shape)
print('----')
print('Test data size and shape', X_test.shape)
print('Test labels size and shape', Y_test.shape)
print('Test metadata size and shape', M_test.shape)

Train data size and shape (4065, 224, 224, 3)
Train labels size and shape (4065,)
Train metadata size and shape (4065, 3)
----
Valid data size and shape (1482, 224, 224, 3)
Valid labels size and shape (1482,)
Valid metadata size and shape (1482, 3)
----
Test data size and shape (1978, 224, 224, 3)
Test labels size and shape (1978,)
Test metadata size and shape (1978, 3)


### Final processing
We are going to use Resnet50

In [ ]:
# train
for i in range(0,X_train.shape[0]):
  x = X_train[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_train[i,] = preprocess_input(x)

# validation
for i in range(0,X_valid.shape[0]):
  x = X_valid[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_valid[i,] = preprocess_input(x)  

# test
for i in range(0,X_test.shape[0]):
  x = X_test[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_test[i,] = preprocess_input(x) 

<a name='2-3'></a>
## The models


In [ ]:
model_1 = False
model_2 = True

In [ ]:
if (model_1):
  skip=False  
  # downloading the data
  #!wget https://data.chalearnlap.cvc.uab.cat/Colab_2021/model.zip

  # decompressing the data
  with ZipFile('drive/MyDrive/Computer Vision/Task 2/model.zip','r') as zip:
    zip.extractall()
    print('Model decompressed successfully')

  # removing the .zip file after extraction  to clean space
  #!rm model.zip

  # loading the pretrained model
  model = tf.keras.models.load_model('./model/weights.h5')

  # Using the FC layer before the 'classifier_low_dim' layer as feature vector
  fc_512 = model.get_layer('dim_proj').output

  # adding a dropout layer to minimize overfiting problems (or not)
  dp_512 = Dropout(0.5)(fc_512)
  fc_64 = Dense(64, activation='relu', name='f_64')(dp_512)

  # adding a few hidden FC layers to learn hidden representations
  #fc_64 = Dense(64, activation='relu', name='f_64')(fc_512)
  fc_16 = Dense(16, activation='relu', name='f_16')(fc_64)
  fc_8 = Dense(8, activation='relu', name='f_8')(fc_16)

  # Includint an additional FC layer with sigmoid activation, used to regress
  # the apparent age
  output = Dense(1, activation='sigmoid', name='predict')(fc_8)

  # building and pringing the final model
  model = Model(inputs=model.get_layer('base_input').output,outputs=output)
  #print(model.summary())

In [ ]:
if (model_2):
  skip=True
  # loading the pretrained model
  model = tf.keras.models.load_model('drive/MyDrive/Computer Vision/Task 2/model_stage1/best_model.h5')

<a name='2-4'></a>
## Training and approach

<a name='2-4-1'></a>
### Phase 1

In [ ]:
patience=4
lr=0.00001
batch=32
epochs=80

In [ ]:
if (not skip):  
  counter = 0
  for layer in model.layers:
    if counter <= 174:   #  using 174 or 133
      layer.trainable = False
    else:
      layer.trainable = True
    # print(counter, layer.name, layer.trainable)
    counter +=1

In [ ]:
if (not skip):  
  # defining the early stop criteria
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)
  # saving the best model based on val_loss
  mc = ModelCheckpoint('/content/drive/MyDrive/temp/best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

  # defining the optimizer
  model.compile(tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.MeanSquaredError(), metrics=['mae']) #loss=tf.keras.losses.MeanSquaredError()loss=tf.keras.losses.Huber()

  # training the model
  history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=batch, epochs=epochs, shuffle=True, verbose=1, callbacks=[es,mc])

  # saving training history (for future visualization)
  with open('/content/drive/MyDrive/temp/train_history.pkl', 'wb') as handle:
    pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

<a name='2-4-2'></a>
### Phase 2

In [ ]:
patience=4
lr=0.00001
batch=16
epochs=25

In [ ]:
if (model_1):
  saved_model = load_model('/content/drive/MyDrive/temp/best_model.h5')
elif (model_2):
  saved_model=model

# setting all layers of the model to trainable
saved_model.trainable = True

counter = 0
for layer in saved_model.layers:
  # print(counter, layer.name, layer.trainable)
  counter +=1

<a name='2-4-3'></a>
### Different group weighted sampling

In [ ]:
og=False
custom_I=False
custom_II=True

<a name='2-4-3-1'></a>
#### Original

In [ ]:
if (og): 
  # counting the number of samples per group in the train data (age attribute only)
  g1 = g2 = g3 = g4 = 0
  for i in range(0,Y_train.shape[0]):
      if(Y_train[i]*100<20):
        g1 +=1
      if(Y_train[i]*100>=20 and Y_train[i]*100<40):
        g2 +=1
      if(Y_train[i]*100>=40 and Y_train[i]*100<60):
        g3 +=1
      if(Y_train[i]*100>=60):
        g4 +=1
  print('group(s) size = ', [g1, g2, g3, g4])

  # generating the weights for each group using the equation defined above
  w = sum(np.array([g1, g2, g3, g4]))/(4*np.array([g1, g2, g3, g4]))
  print('weights per group = ', w)

  # creating a vector with same size as Y_train, that will link a particular label to its weight
  sample_weights = []
  for i in range(0,Y_train.shape[0]):
      if(Y_train[i]*100<20):
        sample_weights.append(w[0])
      if(Y_train[i]*100>=20 and Y_train[i]*100<40):
        sample_weights.append(w[1])
      if(Y_train[i]*100>=40 and Y_train[i]*100<60):
        sample_weights.append(w[2])
      if(Y_train[i]*100>=60):
        sample_weights.append(w[3])
  sample_weights = np.array(sample_weights)

<a name='2-4-3-2'></a>
#### 48 Groups

In [ ]:
if (custom_I):
  #Creating a dictionary
  weights={}
  for i in range(1,49):
    j=str(i)
    weights["g"+j]=0
  # counting the number of samples per group in the train data:
  for i in range(0,Y_train.shape[0]):
      #If young
      if (Y_train[i]*100<20):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g1"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g2"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g3"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g4"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 20 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g5"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g6"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g7"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g8"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 20 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g9"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g10"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g11"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g12"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 20 asian?')
        else:
          raise ValueError('???')
      #If young 40
      elif (Y_train[i]*100<40):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g13"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g14"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g15"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g16"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 40 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g17"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g18"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g19"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g20"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 40 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g21"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g22"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g23"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g24"]+=1
          #ERRORValueError('A very specific bad thing happened.')
          else:
            raise ValueError('No metadata in 40 asian?')
        else:
          raise ValueError('???')
      #If young 60
      elif (Y_train[i]*100<60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g25"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g26"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g27"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g28"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g29"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g30"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g31"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g32"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g33"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g34"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g35"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g36"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 60 asian?')
        else:
          raise ValueError('???')
      #If over 60 40
      elif (Y_train[i]*100>=60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g37"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g38"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g39"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g40"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in over 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["41"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g42"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g43"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g44"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in over 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g45"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g46"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g47"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g48"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in over 60 asian?')
        else:
          raise ValueError('???')
      else:
        raise ValueError('IMPOSIBLE')

  #We have some groups with 0 values
  for i in range(1,49):
    j=str(i)
    if (weights["g"+j]==0):
      weights["g"+j]=1


  #print(weights)
  # generating the weights for each group using the equation defined above
  w = sum(np.array(list(weights.values())))/(48*np.array(list(weights.values())))
  #print('weights per group = ', w)

  # creating a vector with same size as Y_train, that will link a particular label to its weight
  sample_weights = []
  for i in range(0,Y_train.shape[0]):
      #If young
      if (Y_train[i]*100<20):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[0])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[1])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[2])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[3])
          #ERROR
          else:
            raise ValueError('No metadata in 20 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[4])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[5])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[6])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[7])
          #ERROR
          else:
            raise ValueError('No metadata in 20 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[8])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[9])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[10])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[11])
          #ERROR
          else:
            raise ValueError('No metadata in 20 asian?')
        else:
          raise ValueError('???')
      #If young 40
      elif (Y_train[i]*100<40):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[12])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[13])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[14])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[15])
          #ERROR
          else:
            raise ValueError('No metadata in 40 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[16])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[17])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[18])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[19])
          #ERROR
          else:
            raise ValueError('No metadata in 40 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[20])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[21])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
           sample_weights.append(w[22])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[23])
          #ERRORValueError('A very specific bad thing happened.')
          else:
            raise ValueError('No metadata in 40 asian?')
        else:
          raise ValueError('???')
      #If young 60
      elif (Y_train[i]*100<60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[24])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[25])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[26])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[27])
          #ERROR
          else:
            raise ValueError('No metadata in 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[28])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[29])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[30])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[31])
          #ERROR
          else:
            raise ValueError('No metadata in 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[32])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[33])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[34])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[35])
          #ERROR
          else:
            raise ValueError('No metadata in 60 asian?')
        else:
          raise ValueError('???')
      #If over 60 40
      elif (Y_train[i]*100>=60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[36])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[37])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[38])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[39])
          #ERROR
          else:
            raise ValueError('No metadata in over 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[40])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[41])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[42])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[43])
          #ERROR
          else:
            raise ValueError('No metadata in over 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[44])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[45])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[46])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[47])
          #ERROR
          else:
            raise ValueError('No metadata in over 60 asian?')
        else:
          raise ValueError('???')
      else:
        raise ValueError('IMPOSIBLE')
  sample_weights = np.array(sample_weights)

<a name='2-4-3-3'></a>
#### 7 Age groups

In [ ]:
if (custom_II): 
  # counting the number of samples per group in the train data (age attribute only)
  g1 = g2 = g3 = g4 = g5 = g6 = g7 = 0
  for i in range(0,Y_train.shape[0]):
      if(Y_train[i]*100<10):
        g1 +=1
      if(Y_train[i]*100>=10 and Y_train[i]*100<20):
        g2 +=1
      if(Y_train[i]*100>=20 and Y_train[i]*100<30):
        g3 +=1
      if(Y_train[i]*100>=30 and Y_train[i]*100<40):
        g4 +=1
      if(Y_train[i]*100>=40 and Y_train[i]*100<50):
        g5 +=1
      if(Y_train[i]*100>=50 and Y_train[i]*100<60):
        g6 +=1
      if(Y_train[i]*100>=60):
        g7 +=1

  # generating the weights for each group using the equation defined above
  w = sum(np.array([g1, g2, g3, g4, g5, g6, g7]))/(7*np.array([g1, g2, g3, g4, g5, g6, g7]))
  #print('weights per group = ', w)

  # creating a vector with same size as Y_train, that will link a particular label to its weight
  sample_weights = []
  for i in range(0,Y_train.shape[0]):
      if(Y_train[i]*100<10):
        sample_weights.append(w[0])
      if(Y_train[i]*100>=10 and Y_train[i]*100<20):
        sample_weights.append(w[1])
      if(Y_train[i]*100>=20 and Y_train[i]*100<30):
        sample_weights.append(w[2])
      if(Y_train[i]*100>=30 and Y_train[i]*100<40):
        sample_weights.append(w[3])
      if(Y_train[i]*100>=40 and Y_train[i]*100<50):
        sample_weights.append(w[4])
      if(Y_train[i]*100>=50 and Y_train[i]*100<60):
        sample_weights.append(w[5])
      if(Y_train[i]*100>=60):
        sample_weights.append(w[6])
  sample_weights = np.array(sample_weights)

<a name='3'></a>
#Custom losses Research

<a name='3-1'></a>
## Pre-requisites

In [ ]:
!pip install tensorflow
!pip install opencv-python
!pip install h5py

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<a name='3-2'></a>
## Imports

In [ ]:
import h5py
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt
import random 
import pickle
import cv2
import numpy as np

<a name='3-3'></a>
## Data Gathering

### Loading the train/validation data, and re-scaling the labels to [0..1]

In [ ]:
# downloading the data
#!wget https://data.chalearnlap.cvc.uab.cat/Colab_2021/app_data.zip

# decompressing the data
from zipfile import ZipFile

with ZipFile('drive/MyDrive/app_data.zip','r') as zip:
   zip.extractall()
   print('Data decompressed successfully')

# removing the .zip file after extraction to clean space
#!rm app_data.zip

In [ ]:
import numpy as np

# loading the train data
X_train = np.load('./data/data_train.npy')
Y_train = np.load('./data/labels_train.npy')
M_train = np.load('./data/meta_data_train.npy')

# loading the validation data
X_valid = np.load('./data/data_valid.npy')
Y_valid = np.load('./data/labels_valid.npy')
M_valid = np.load('./data/meta_data_valid.npy')

# loading the test data
X_test = np.load('./data/data_test.npy')
Y_test = np.load('./data/labels_test.npy')
M_test = np.load('./data/meta_data_test.npy')


Y_train = Y_train/100
Y_valid = Y_valid/100
# Y_test = Y_test/100 

print('Train data size and shape', X_train.shape)
print('Train labels size and shape', Y_train.shape)
print('Train metadata size and shape', M_train.shape)
print('----')
print('Valid data size and shape', X_valid.shape)
print('Valid labels size and shape', Y_valid.shape)
print('Valid metadata size and shape', M_valid.shape)
print('----')
print('Test data size and shape', X_test.shape)
print('Test labels size and shape', Y_test.shape)
print('Test metadata size and shape', M_test.shape)

### Preprocessing the data (face images)

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

# train
for i in range(0,X_train.shape[0]):
  x = X_train[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_train[i,] = preprocess_input(x)

# validation
for i in range(0,X_valid.shape[0]):
  x = X_valid[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_valid[i,] = preprocess_input(x)  

# test
for i in range(0,X_test.shape[0]):
  x = X_test[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_test[i,] = preprocess_input(x) 

<a name='3-4'></a>
## The Task 1 model

### Downloading the ResNet50 model pre-trained on Faces

In [ ]:
# downloading the data
!wget https://data.chalearnlap.cvc.uab.cat/Colab_2021/model.zip

# decompressing the data
with ZipFile('drive/MyDrive/model.zip','r') as zip:
   zip.extractall()
   print('Model decompressed successfully')

# removing the .zip file after extraction  to clean space
!rm model.zip

### Loading the pre-trained model

In [ ]:
# loading the pretrained model
model = tf.keras.models.load_model('./model/weights.h5')

# print the model summary
#print(model.summary())

In [ ]:
# Using the FC layer before the 'classifier_low_dim' layer as feature vector
fc_512 = model.get_layer('dim_proj').output

# adding a dropout layer to minimize overfiting problems
#dp_layer = Dropout(0.5)(fc_512)

# adding a few hidden FC layers to learn hidden representations
#fc_128 = Dense(128, activation='relu', name='f_128')(fc_512)
#fc_32 = Dense(32, activation='relu', name='f_32')(fc_128)

fc_64 = Dense(64, activation='relu', name='f_64')(fc_512)
fc_16 = Dense(16, activation='relu', name='f_16')(fc_64)
fc_8 = Dense(8, activation='relu', name='f_8')(fc_16)

# Includint an additional FC layer with sigmoid activation, used to regress
# the apparent age
output = Dense(1, activation='sigmoid', name='predict')(fc_8)

# building and pringing the final model
model = Model(inputs=model.get_layer('base_input').output,outputs=output)
#print(model.summary())

### Using the best parameters, model  architecture achieved in task 1 to train the resenet50 for stage 1

In [ ]:
counter = 0
for layer in model.layers:
  if counter <= 174: 
    layer.trainable = False
  else:
    layer.trainable = True
  print(counter, layer.name, layer.trainable)
  counter +=1
#print(model.summary())


# defining the early stop criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# saving the best model based on val_loss
mc = ModelCheckpoint('/content/gdrive/MyDrive/temp/best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

# defining the optimizer
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5),loss=tf.keras.losses.MeanSquaredError(),metrics=['mae'])

# training the model
history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=32, epochs=30, shuffle=True, verbose=1, callbacks=[es,mc])

# saving training history (for future visualization)
with open('/content/gdrive/MyDrive/temp/train_history.pkl', 'wb') as handle:
  pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)


saved_model = load_model('/content/gdrive/MyDrive/temp/best_model.h5')


# predict on the test data
predictions_st1 = saved_model.predict(X_test, batch_size=32, verbose=1)

predictions_st1_f = predictions_st1*100

# evaluating on test data
error = []
for i in range(0,len(Y_test)):
  error.append(abs(np.subtract(predictions_st1_f[i][0],Y_test[i])))

print('MAE = %.8f' %(np.mean(error)))

for i in range(0,10):
    print('predicted age = %.3f - Ground truth = %.3f' %(predictions_st1_f[i], Y_test[i]))

### Using the best parameters, model  architecture achieved in task 1 to train the resenet50 for stage 2 without custom loss

In [ ]:
# setting all layers of the model to trainable
saved_model.trainable = True

counter = 0
for layer in saved_model.layers:
  print(counter, layer.name, layer.trainable)
  counter +=1

# training all layers (2nd stage), given the model saved on stage 1
saved_model.compile(tf.keras.optimizers.Adam(learning_rate=5e-5),loss=tf.keras.losses.MeanSquaredError(),metrics=['mae'])

# defining the early stop criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# saving the best model (2nd stage) based on val_loss with a different name
mc = ModelCheckpoint('/content/gdrive/MyDrive/temp/best_model_2nd_stage.h5', monitor='val_loss', mode='min', save_best_only=True)

history = saved_model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=16, epochs=10, shuffle=True, verbose=1, callbacks=[es,mc])

# saving training history
with open('/content/gdrive/MyDrive/temp/train_history_2nd_stage.pkl', 'wb') as handle:
  pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)

train_hist = pickle.load(open('/content/gdrive/MyDrive/temp/train_history.pkl',"rb"))
train_hist_2nd = pickle.load(open('/content/gdrive/MyDrive/temp/train_history_2nd_stage.pkl',"rb"))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
fig.suptitle('Training history (Stage 1 and Stage 2)', fontsize=14, fontweight='bold')

ax1.plot(train_hist['loss']+train_hist_2nd['loss'])
ax1.plot(train_hist['val_loss']+train_hist_2nd['val_loss'])
ax1.axvline(42, 0, 1, ls='--', color='r')
ax1.set(xlabel='epoch', ylabel='Loss')
ax1.legend(['train', 'valid'], loc='upper right')

ax2.plot(train_hist['mae']+train_hist_2nd['mae'])
ax2.plot(train_hist['val_mae']+train_hist_2nd['val_mae'])
ax2.axvline(42, 0, 1, ls='--', color='r')
ax2.set(xlabel='epoch', ylabel='MAE')
ax2.legend(['train', 'valid'], loc='upper right')

saved_model_2nd = load_model('/content/gdrive/MyDrive/temp/best_model_2nd_stage.h5')



# predict on the test data
predictions_2nd = saved_model_2nd.predict(X_test, batch_size=32, verbose=1)
predictions_2nd_f = predictions_2nd*100
error = []
for i in range(0,len(Y_test)):
  error.append(abs(np.subtract(predictions_2nd_f[i][0],Y_test[i])))

print('MAE = %.8f' %(np.mean(error)))
# printing some predictions
for i in range(0,10):
  print('predicted age = %.3f - Ground truth = %.3f' %(predictions_2nd_f[i], Y_test[i]))



<a name='3-5'></a>
## Improving Task 1 results using phase 2 as baseline

<a name='3-5-1'></a>
### Load the Train data again  and generate the weigths 

We are only going to use the first two sample weights

In [ ]:
# loading the train data again (original face images, before preprocessing):
X_train = np.load('./data/data_train.npy')
Y_train = np.load('./data/labels_train.npy')
Y_train = Y_train/100 # normalizing the age values to be between [0,1]

# preprocessing the train data with respect to ResNet-50 Inputs.
for i in range(0,X_train.shape[0]):
  x = X_train[i,:,:,:]
  x = np.expand_dims(x, axis=0)
  X_train[i,] = preprocess_input(x)

# counting the number of samples per group in the train data (age attribute only)
g1 = g2 = g3 = g4 = 0
for i in range(0,Y_train.shape[0]):
    if(Y_train[i]*100<20):
      g1 +=1
    if(Y_train[i]*100>=20 and Y_train[i]*100<40):
      g2 +=1
    if(Y_train[i]*100>=40 and Y_train[i]*100<60):
      g3 +=1
    if(Y_train[i]*100>=60):
      g4 +=1
print('group(s) size = ', [g1, g2, g3, g4])

# generating the weights for each group using the equation defined above
w = sum(np.array([g1, g2, g3, g4]))/(4*np.array([g1, g2, g3, g4]))
print('weights per group = ', w)

# creating a vector with same size as Y_train, that will link a particular label to its weight
sample_weights = []
for i in range(0,Y_train.shape[0]):
    if(Y_train[i]*100<20):
      sample_weights.append(w[0])
    if(Y_train[i]*100>=20 and Y_train[i]*100<40):
      sample_weights.append(w[1])
    if(Y_train[i]*100>=40 and Y_train[i]*100<60):
      sample_weights.append(w[2])
    if(Y_train[i]*100>=60):
      sample_weights.append(w[3])
sample_weights = np.array(sample_weights)


In [ ]:
custome_I=True
if custom_I=True:
  #Creating a dictionary
  weights={}
  for i in range(1,49):
    j=str(i)
    weights["g"+j]=0
  # counting the number of samples per group in the train data:
  for i in range(0,Y_train.shape[0]):
      #If young
      if (Y_train[i]*100<20):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g1"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g2"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g3"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g4"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 20 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g5"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g6"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g7"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g8"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 20 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g9"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g10"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g11"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g12"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 20 asian?')
        else:
          raise ValueError('???')
      #If young 40
      elif (Y_train[i]*100<40):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g13"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g14"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g15"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g16"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 40 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g17"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g18"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g19"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g20"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 40 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g21"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g22"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g23"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g24"]+=1
          #ERRORValueError('A very specific bad thing happened.')
          else:
            raise ValueError('No metadata in 40 asian?')
        else:
          raise ValueError('???')
      #If young 60
      elif (Y_train[i]*100<60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g25"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g26"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g27"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g28"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g29"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g30"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g31"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g32"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g33"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g34"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g35"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g36"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in 60 asian?')
        else:
          raise ValueError('???')
      #If over 60 40
      elif (Y_train[i]*100>=60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g37"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g38"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g39"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g40"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in over 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["41"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g42"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g43"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g44"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in over 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            weights["g45"]+=1
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            weights["g46"]+=1
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            weights["g47"]+=1
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            weights["g48"]+=1
          #ERROR
          else:
            raise ValueError('No metadata in over 60 asian?')
        else:
          raise ValueError('???')
      else:
        raise ValueError('IMPOSIBLE')

  #We have some groups with 0 values
  for i in range(1,49):
    j=str(i)
    if (weights["g"+j]==0):
      weights["g"+j]=1


  #print(weights)
  # generating the weights for each group using the equation defined above
  w = sum(np.array(list(weights.values())))/(48*np.array(list(weights.values())))
  #print('weights per group = ', w)

  # creating a vector with same size as Y_train, that will link a particular label to its weight
  sample_weights = []
  for i in range(0,Y_train.shape[0]):
      #If young
      if (Y_train[i]*100<20):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[0])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[1])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[2])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[3])
          #ERROR
          else:
            raise ValueError('No metadata in 20 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[4])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[5])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[6])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[7])
          #ERROR
          else:
            raise ValueError('No metadata in 20 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[8])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[9])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[10])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[11])
          #ERROR
          else:
            raise ValueError('No metadata in 20 asian?')
        else:
          raise ValueError('???')
      #If young 40
      elif (Y_train[i]*100<40):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[12])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[13])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[14])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[15])
          #ERROR
          else:
            raise ValueError('No metadata in 40 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[16])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[17])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[18])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[19])
          #ERROR
          else:
            raise ValueError('No metadata in 40 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[20])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[21])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
           sample_weights.append(w[22])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[23])
          #ERRORValueError('A very specific bad thing happened.')
          else:
            raise ValueError('No metadata in 40 asian?')
        else:
          raise ValueError('???')
      #If young 60
      elif (Y_train[i]*100<60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[24])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[25])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[26])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[27])
          #ERROR
          else:
            raise ValueError('No metadata in 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[28])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[29])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[30])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[31])
          #ERROR
          else:
            raise ValueError('No metadata in 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[32])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[33])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[34])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[35])
          #ERROR
          else:
            raise ValueError('No metadata in 60 asian?')
        else:
          raise ValueError('???')
      #If over 60 40
      elif (Y_train[i]*100>=60):
        #If caucassian
        if (str(M_train[i][1]).lower()=="caucasian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[36])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[37])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[38])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[39])
          #ERROR
          else:
            raise ValueError('No metadata in over 60 caucasian?')
        elif (str(M_train[i][1]).lower()=="afroamerican"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[40])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[41])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[42])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[43])
          #ERROR
          else:
            raise ValueError('No metadata in over 60 afroamerican?')
        elif (str(M_train[i][1]).lower()=="asian"):
          #If neutral
          if (str(M_train[i][2]).lower()=="neutral"):
            sample_weights.append(w[44])
          #If slightly happy
          elif (str(M_train[i][2]).lower()=="slightlyhappy"):
            sample_weights.append(w[45])
          #If happy
          elif (str(M_train[i][2]).lower()=="happy"):
            sample_weights.append(w[46])
          #If other
          elif (str(M_train[i][2]).lower()=="other"):
            sample_weights.append(w[47])
          #ERROR
          else:
            raise ValueError('No metadata in over 60 asian?')
        else:
          raise ValueError('???')
      else:
        raise ValueError('IMPOSIBLE')
  sample_weights = np.array(sample_weights)

<a name='3-5-2'></a>
### Define the custom losses
Below are the implementations of the weighted focal mse loss, the weighted huber loss and the weighted quantile loss.

In [ ]:
def weighted_focal_mse_loss(y_true, y_pred, alpha=1., gamma=2, weights=sample_weights):
    #loss = tf.square(y_true - y_pred) 
    
    loss =alpha*tf.keras.backend.pow(tf.square(y_true - y_pred),gamma) 
    if weights is not None:
        loss *= weights
    loss = tf.keras.backend.mean(loss)
    
    return loss

In [ ]:
def weighted_huber_loss(y_true, y_pred,  delta=1, weights=sample_weights):
    l1_loss = tf.keras.backend.abs(y_true - y_pred)

    loss = tf.where(l1_loss < delta, 0.5 * (l1_loss ** 2), 0.5 * delta**2 + delta * (l1_loss - delta))
    if weights is not None:
        loss *= weights
    loss = tf.keras.backend.mean(loss)
    return loss

In [ ]:
def weighted_quantile_loss(y_true, y_pred,  gamma=0.7, weights=sample_weights):
    #l1_loss = tf.keras.backend.abs(y_true - y_pred)
    
    loss=gamma*tf.keras.backend.maximum(0.,y_true - y_pred)+(1-gamma)*tf.keras.backend.maximum(0.,y_pred-y_true)
    
    if weights is not None:
        loss *= weights
    loss = tf.keras.backend.mean(loss)
    return loss


<a name='3-5-3'></a>
### Using the sample weights and custom loss to train our model in phase 2


In [ ]:


NUM_EPOCHS = 10

RESUME_TRAINING = False


if(RESUME_TRAINING == False):
  saved_model = load_model('/content/gdrive/MyDrive/temp/best_model.h5') # load model from stage 1
else:
  # resume training (stage 2)
  saved_model = load_model('/content/gdrive/MyDrive/temp/best_model_2nd_stage_weighted.h5')
  
# setting all layers to traineble
saved_model.trainable = True

#=================================================
# training all layers (2nd stage), given the model saved on stage 1
#saved_model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5),loss=tf.keras.losses.MeanSquaredError(),metrics=['mae'])
saved_model.compile(tf.keras.optimizers.Adam(learning_rate=5e-5),loss=weighted_huber_loss,metrics=['mae'])
#=================================================

# defining the early stop criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('/content/gdrive/MyDrive/temp/best_model_2nd_stage_weighted.h5', monitor='val_loss', mode='min', save_best_only=True)

if(RESUME_TRAINING == False):  
  #history = saved_model.fit(X_train, Y_train, sample_weight=sample_weights, validation_data=(X_valid, Y_valid), batch_size=16, epochs=NUM_EPOCHS, initial_epoch=0, shuffle=True, verbose=1, callbacks=[es,mc])
  history = saved_model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=16, epochs=NUM_EPOCHS, initial_epoch=0, shuffle=True, verbose=1, callbacks=[es,mc])
  
else:

  #history = saved_model.fit(X_train, Y_train, sample_weight=sample_weights, validation_data=(X_valid, Y_valid), batch_size=16, epochs=NUM_EPOCHS, initial_epoch=RESUME_FROM_EPOCH, shuffle=True, verbose=1, callbacks=[es,mc])
  history = saved_model.fit(X_train, Y_train,  validation_data=(X_valid, Y_valid), batch_size=16, epochs=NUM_EPOCHS, initial_epoch=RESUME_FROM_EPOCH, shuffle=True, verbose=1, callbacks=[es,mc])
# saving training history
with open('/content/gdrive/MyDrive/temp/train_history_2nd_stage_weighted.pkl', 'wb') as handle:
  pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
  